## Microtask0

### Problem Statement:
Use this notebook implementing the Code_Changes metric (see it in MyBinder) as an example of how to collect the data, producing a single JSON file per data source, with all items (commits, issues, pull/merge requests) in it. Produce one notebook per data source (git, GitHub/GitLab issues, GitHub pull requests / GitLab merge requests) showing a summary of the contents of that file (number of items in it, and number of different identities in it counting authors/committers for git, submitters for issues and pull/merge requests). This microtask is mandatory, to show that you can retrieve data and produde a notebook showing it. In each notebook, include also the list of repositories retrieved, and the date of retrieval, using data available in the JSON file.

### Project: django

### Repositories

In [5]:
import json
import datetime
from collections import defaultdict
from pprint import pprint


### Getting the data

In [23]:
# github_url = "https://github.com/"
# owner = "sarthak-sehgal"
# repos_used = ["timetable-visualizer"]
github_url = "https://github.com/"
owner = "django"
repos_used = ["channels"]
repo_urls = [github_url + owner + "/" + repo_used for repo_used in repos_used]
auth_token = "f4d98f2cef0a0b4873b723fe3ee4574765e8b37f"
print(repo_urls)



['https://github.com/django/channels']


In [24]:
for repo, repo_url in zip(repos_used, repo_urls):
    print(repo, repo_url)
    !perceval git --json-line $repo_url >> trial.json
    print("...")
    !perceval github -t $auth_token --json-line --sleep-for-rate --category pull_request $owner $repo >> trial.json
    print("...")
    !perceval github -t $auth_token --json-line --sleep-for-rate --category issue $owner $repo >> trial.json
    print("...")

channels https://github.com/django/channels
[2019-03-18 20:35:03,875] - Sir Perceval is on his quest.
[2019-03-18 20:37:08,273] - Fetching commits: 'https://github.com/django/channels' git repository from 1970-01-01 00:00:00+00:00 to 2100-01-01 00:00:00+00:00; all branches
[2019-03-18 20:37:13,075] - Fetch process completed: 1072 commits fetched
[2019-03-18 20:37:13,077] - Sir Perceval completed his quest.
...
[2019-03-18 20:37:13,639] - Sir Perceval is on his quest.
[2019-03-18 20:37:23,771] - Getting info for https://api.github.com/users/nemesisdesign
[2019-03-18 20:37:26,065] - Getting info for https://api.github.com/users/andrewgodwin
[2019-03-18 20:37:33,739] - Getting info for https://api.github.com/users/hawkowl
[2019-03-18 20:37:38,054] - Getting info for https://api.github.com/users/hirokiky
[2019-03-18 20:37:44,528] - Getting info for https://api.github.com/users/octaflop
[2019-03-18 20:37:50,300] - Getting info for https://api.github.com/users/asmaps
[2019-03-18 20:37:56,944

[2019-03-18 20:47:42,593] - Getting info for https://api.github.com/users/scryver
[2019-03-18 20:47:45,728] - Getting info for https://api.github.com/users/askoretskiy
[2019-03-18 20:47:48,989] - Getting info for https://api.github.com/users/koole
[2019-03-18 20:47:51,627] - Getting info for https://api.github.com/users/devmapal
[2019-03-18 20:47:55,338] - Getting info for https://api.github.com/users/simonw
[2019-03-18 20:47:57,708] - Getting info for https://api.github.com/users/muslu
[2019-03-18 20:48:00,346] - Getting info for https://api.github.com/users/mcallistersean
[2019-03-18 20:48:03,107] - Getting info for https://api.github.com/users/raphael-boucher
[2019-03-18 20:48:25,863] - Getting info for https://api.github.com/users/vartec
[2019-03-18 20:48:38,459] - Getting info for https://api.github.com/users/proofit404
[2019-03-18 20:48:41,322] - Getting info for https://api.github.com/users/germanjoey
[2019-03-18 20:48:46,135] - Getting info for https://api.github.com/users/good

[2019-03-18 21:00:21,530] - Getting info for https://api.github.com/users/Bartzi
[2019-03-18 21:00:24,417] - Getting info for https://api.github.com/users/JuanjoA
[2019-03-18 21:00:32,358] - Getting info for https://api.github.com/users/martoncsikos
[2019-03-18 21:00:37,828] - Getting info for https://api.github.com/users/tiltec
[2019-03-18 21:00:41,694] - Getting info for https://api.github.com/users/ruralscenery
[2019-03-18 21:00:48,162] - Getting info for https://api.github.com/users/superposition
[2019-03-18 21:00:54,513] - Getting info for https://api.github.com/users/MattBroach
[2019-03-18 21:01:06,598] - Getting info for https://api.github.com/users/vrubenjn
[2019-03-18 21:01:09,466] - Getting info for https://api.github.com/users/ZijianHe
[2019-03-18 21:01:11,728] - Getting info for https://api.github.com/users/Sgiath
[2019-03-18 21:01:15,652] - Getting info for https://api.github.com/users/ross-weir
[2019-03-18 21:01:21,305] - Getting info for https://api.github.com/users/xinb

[2019-03-18 21:06:53,133] - Getting info for https://api.github.com/users/mikekeda
[2019-03-18 21:06:53,991] - Getting info for https://api.github.com/users/daxaxelrod
[2019-03-18 21:06:57,496] - Getting info for https://api.github.com/users/evansd
[2019-03-18 21:07:00,178] - Getting info for https://api.github.com/users/screx
[2019-03-18 21:07:01,892] - Getting info for https://api.github.com/users/benjaoming
[2019-03-18 21:07:05,250] - Getting info for https://api.github.com/users/collinanderson
[2019-03-18 21:07:08,267] - Getting info for https://api.github.com/users/darklow
[2019-03-18 21:07:10,072] - Getting info for https://api.github.com/users/gregmuellegger
[2019-03-18 21:07:17,423] - Getting info for https://api.github.com/users/michaelkuty
[2019-03-18 21:07:20,428] - Getting info for https://api.github.com/users/vracheva
[2019-03-18 21:07:22,958] - Getting info for https://api.github.com/users/exp0nge
[2019-03-18 21:07:24,100] - Getting info for https://api.github.com/users/d

[2019-03-18 21:11:31,116] - Getting info for https://api.github.com/users/yaslim
[2019-03-18 21:11:33,219] - Getting info for https://api.github.com/users/BuckinghamIO
[2019-03-18 21:11:38,874] - Getting info for https://api.github.com/users/anton-white
[2019-03-18 21:11:40,852] - Getting info for https://api.github.com/users/pirate
[2019-03-18 21:11:44,128] - Getting info for https://api.github.com/users/zachmccormick
[2019-03-18 21:11:46,168] - Getting info for https://api.github.com/users/Skorpyon
[2019-03-18 21:11:47,649] - Getting info for https://api.github.com/users/mwaaas
[2019-03-18 21:11:53,358] - Getting info for https://api.github.com/users/ludvigsnn
[2019-03-18 21:11:54,797] - Getting info for https://api.github.com/users/byoungb
[2019-03-18 21:11:56,139] - Getting info for https://api.github.com/users/vinay13
[2019-03-18 21:12:03,926] - Getting info for https://api.github.com/users/tysonclugg
[2019-03-18 21:12:05,671] - Getting info for https://api.github.com/users/furiou

[2019-03-18 21:17:07,283] - Getting info for https://api.github.com/users/carltongibson
[2019-03-18 21:17:08,996] - Getting info for https://api.github.com/users/rctoris
[2019-03-18 21:17:11,470] - Getting info for https://api.github.com/users/imaurer
[2019-03-18 21:17:17,534] - Getting info for https://api.github.com/users/mTatcher
[2019-03-18 21:17:18,974] - Getting info for https://api.github.com/users/ericholscher
[2019-03-18 21:17:22,292] - Getting info for https://api.github.com/users/zeezdev
[2019-03-18 21:17:24,837] - Getting info for https://api.github.com/users/vanikin
[2019-03-18 21:17:26,027] - Getting info for https://api.github.com/users/CptLemming
[2019-03-18 21:17:30,975] - Getting info for https://api.github.com/users/rixx
[2019-03-18 21:17:33,967] - Getting info for https://api.github.com/users/hoaivan
[2019-03-18 21:17:36,238] - Getting info for https://api.github.com/users/daybarr
[2019-03-18 21:17:37,156] - Getting info for https://api.github.com/users/Copser
[2019

[2019-03-18 21:21:47,927] - Getting info for https://api.github.com/users/yanikkoval
[2019-03-18 21:21:53,690] - Getting info for https://api.github.com/users/rexlertech
[2019-03-18 21:21:58,986] - Getting info for https://api.github.com/users/ErwinJunge
[2019-03-18 21:22:04,779] - Getting info for https://api.github.com/users/Arudan
[2019-03-18 21:22:07,374] - Getting info for https://api.github.com/users/freehugs
[2019-03-18 21:22:12,727] - Getting info for https://api.github.com/users/theY4Kman
[2019-03-18 21:22:15,762] - Getting info for https://api.github.com/users/Zacharilius
[2019-03-18 21:22:19,310] - Getting info for https://api.github.com/users/last-partizan
[2019-03-18 21:22:21,247] - Getting info for https://api.github.com/users/mahrous78
[2019-03-18 21:22:23,359] - Getting info for https://api.github.com/users/adriancarayol
[2019-03-18 21:22:25,143] - Getting info for https://api.github.com/users/lingliqianxun
[2019-03-18 21:22:28,648] - Getting info for https://api.github

[2019-03-18 21:27:02,859] - Getting info for https://api.github.com/users/hugobrilhante
[2019-03-18 21:27:04,605] - Getting info for https://api.github.com/users/atbaker
[2019-03-18 21:27:06,383] - Getting info for https://api.github.com/users/kmctown
[2019-03-18 21:27:10,197] - Getting info for https://api.github.com/users/Braintelligence
[2019-03-18 21:27:13,173] - Getting info for https://api.github.com/users/Hebertl
[2019-03-18 21:27:15,292] - Getting info for https://api.github.com/users/void4
[2019-03-18 21:27:16,807] - Getting info for https://api.github.com/users/slact
[2019-03-18 21:27:19,093] - Getting info for https://api.github.com/users/johnnyflute
[2019-03-18 21:27:20,257] - Getting info for https://api.github.com/users/puterleat
[2019-03-18 21:27:21,105] - Getting info for https://api.github.com/users/benwhalley
[2019-03-18 21:27:23,252] - Getting info for https://api.github.com/users/TheBlusky
[2019-03-18 21:27:25,117] - Getting info for https://api.github.com/users/arm

In [18]:
class Code_Changes:
               
    
    def __init__(self, path_to_file):
        
        # this will emulate a pandas dataframe 
        self.commits_data = defaultdict(list)        
        
        with open(path_to_file) as jsonfile:
            
            for line in jsonfile:
                
                # the commit_breakdown method is designed for commits only
                line = json.loads(line)
                if line['category'] == "commit":
                    commit = line
                    
                    commit_breakdown = self._get_commit_breakdown(commit)
                    
                    for col, val in commit_breakdown.items():
                        
                        self.commits_data[col].append(val)
                        print(".", end=' ')

        
    @staticmethod
    def _get_commit_breakdown(commit):
        repo_name = commit['origin']
        repo_data = commit['data']
        
        commit_breakdown = {
            "repo_name": repo_name,
            "hash": repo_data['commit'],
            "commit_auth": repo_data['Author'],
            "author_date": datetime.datetime.strptime(repo_data['AuthorDate'],"%a %b %d %H:%M:%S %Y %z"),
            'commit': repo_data['Commit'],
            'commit_date': datetime.datetime.strptime(repo_data['CommitDate'],"%a %b %d %H:%M:%S %Y %z"),
            'files_no': len(repo_data['files'])
        }
        
        actions = 0
        for file in repo_data['files']:
            if 'action' in file:
                actions += 1
                
        commit_breakdown['files_action'] = actions
        
        if 'Merge' in repo_data:
            commit_breakdown['merge'] = True
        else:
            commit_breakdown['merge'] = False
            
        return commit_breakdown
        
    
    
    def number_of_repos(self):
        return len(repos_used)
    
    def total_commits(self):
        return len(self.commits_data)
    
    def total_commits_per_repo(self):
        commits_per_repo = dict()
        
        for repo in commits_data['repo_name']:
            commits_per_repo[repo] += 1
    
        return commits_per_repo
    
    def count_from_to(self, start=None, end=None, type_of_date="author_date"):
        pass
        
    

# Analyzing the data using the Class

In [19]:
commits_data = Code_Changes('trial.json')

hi []
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

## Total number of commits 

In [20]:
print("The total number of commits in all repos is: ", commits_data.total_commits())

for repo in repos_used:
    print("The number of commits in {0} are {1}".format(repo, commits_data.total_commits_per_repo(repo)))

The total number of commits in all repos is:  10


TypeError: total_commits_per_repo() takes 1 positional argument but 2 were given

## Total number of commits between dates

In [21]:
print("Code changes count all period:", commits_data.count())
print("Code changes count from 2018-01-01 to 2018-07-01:",
      commits_data.count(since="2018-01-01", until="2018-07-01"))
print("Code changes count from 2018-01-01 to 2018-07-01 (no merge commits):",
      commits_data.count(since="2018-01-01", until="2018-07-01", merge=False))
print("Code changes count from 2018-01-01 to 2018-07-01 (no empty commits):",
      commits_data.count(since="2018-01-01", until="2018-07-01", empty=False))

AttributeError: 'Code_Changes' object has no attribute 'count'

# Analyzing the json file directly

In [22]:
github_data = defaultdict(list)

count = 0
with open('trial.json', 'r') as github_data_file:
    for line in github_data_file:
        
            data_line = json.loads(line)
            print(count)
            count += 1

            category = data_line['category']
            data = data_line['data']
            github_data[category].append(data)
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


## Total number of commits in the master branch


In [ ]:
pass

## Total number of non empty commits

## Total number of merge commits

## Total number of  non-empty commits in the master branch

# Pull Requests and Issues

## Total number of pull requests and issues

In [13]:
total_issues = len(github_data["issue"])

print("The number of issues is {0}".format(total_issues))
print("The number of pull requests is {0}".format(len(github_data["pull_request"])))

The number of issues is 1
The number of pull requests is 0


## Total number of open and closed issues

In [14]:
num_open_issues = 0
for issue in github_data["issue"]:
    if issue['state'] == open:
        num_open_issues += 1
        
print("The number of open issues is ", num_open_issues)
print("The number of closed issues is ", total_issues - num_open_issues)

The number of open issues is  0
The number of closed issues is  1


## Number of unique contributers

In [15]:
a = '{"backend_name":"GitHub","backend_version":"0.17.4","category":"pull_request","data":{"_links":{"comments":{"href":"https://api.github.com/repos/django/djangoproject.com/issues/17/comments"},"commits":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/17/commits"},"html":{"href":"https://github.com/django/djangoproject.com/pull/17"},"issue":{"href":"https://api.github.com/repos/django/djangoproject.com/issues/17"},"review_comment":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/comments{/number}"},"review_comments":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/17/comments"},"self":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/17"},"statuses":{"href":"https://api.github.com/repos/django/djangoproject.com/statuses/f397b881184a14510fb5a8ef1dd72732839fd286"}},"additions":1,"assignee":null,"assignees":[],"author_association":"CONTRIBUTOR","base":{"label":"django:master","ref":"master","repo":{"archive_url":"https://api.github.com/repos/django/djangoproject.com/{archive_format}{/ref}","archived":false,"assignees_url":"https://api.github.com/repos/django/djangoproject.com/assignees{/user}","blobs_url":"https://api.github.com/repos/django/djangoproject.com/git/blobs{/sha}","branches_url":"https://api.github.com/repos/django/djangoproject.com/branches{/branch}","clone_url":"https://github.com/django/djangoproject.com.git","collaborators_url":"https://api.github.com/repos/django/djangoproject.com/collaborators{/collaborator}","comments_url":"https://api.github.com/repos/django/djangoproject.com/comments{/number}","commits_url":"https://api.github.com/repos/django/djangoproject.com/commits{/sha}","compare_url":"https://api.github.com/repos/django/djangoproject.com/compare/{base}...{head}","contents_url":"https://api.github.com/repos/django/djangoproject.com/contents/{+path}","contributors_url":"https://api.github.com/repos/django/djangoproject.com/contributors","created_at":"2010-05-28T11:27:09Z","default_branch":"master","deployments_url":"https://api.github.com/repos/django/djangoproject.com/deployments","description":"Source code to djangoproject.com","downloads_url":"https://api.github.com/repos/django/djangoproject.com/downloads","events_url":"https://api.github.com/repos/django/djangoproject.com/events","fork":false,"forks":478,"forks_count":478,"forks_url":"https://api.github.com/repos/django/djangoproject.com/forks","full_name":"django/djangoproject.com","git_commits_url":"https://api.github.com/repos/django/djangoproject.com/git/commits{/sha}","git_refs_url":"https://api.github.com/repos/django/djangoproject.com/git/refs{/sha}","git_tags_url":"https://api.github.com/repos/django/djangoproject.com/git/tags{/sha}","git_url":"git://github.com/django/djangoproject.com.git","has_downloads":true,"has_issues":true,"has_pages":false,"has_projects":false,"has_wiki":false,"homepage":"https://www.djangoproject.com/","hooks_url":"https://api.github.com/repos/django/djangoproject.com/hooks","html_url":"https://github.com/django/djangoproject.com","id":690903,"issue_comment_url":"https://api.github.com/repos/django/djangoproject.com/issues/comments{/number}","issue_events_url":"https://api.github.com/repos/django/djangoproject.com/issues/events{/number}","issues_url":"https://api.github.com/repos/django/djangoproject.com/issues{/number}","keys_url":"https://api.github.com/repos/django/djangoproject.com/keys{/key_id}","labels_url":"https://api.github.com/repos/django/djangoproject.com/labels{/name}","language":"PostScript","languages_url":"https://api.github.com/repos/django/djangoproject.com/languages","license":{"key":"other","name":"Other","node_id":"MDc6TGljZW5zZTA=","spdx_id":"NOASSERTION","url":null},"merges_url":"https://api.github.com/repos/django/djangoproject.com/merges","milestones_url":"https://api.github.com/repos/django/djangoproject.com/milestones{/number}","mirror_url":null,"name":"djangoproject.com","node_id":"MDEwOlJlcG9zaXRvcnk2OTA5MDM=","notifications_url":"https://api.github.com/repos/django/djangoproject.com/notifications{?since,all,participating}","open_issues":46,"open_issues_count":46,"owner":{"avatar_url":"https://avatars2.githubusercontent.com/u/27804?v=4","events_url":"https://api.github.com/users/django/events{/privacy}","followers_url":"https://api.github.com/users/django/followers","following_url":"https://api.github.com/users/django/following{/other_user}","gists_url":"https://api.github.com/users/django/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/django","id":27804,"login":"django","node_id":"MDEyOk9yZ2FuaXphdGlvbjI3ODA0","organizations_url":"https://api.github.com/users/django/orgs","received_events_url":"https://api.github.com/users/django/received_events","repos_url":"https://api.github.com/users/django/repos","site_admin":false,"starred_url":"https://api.github.com/users/django/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/django/subscriptions","type":"Organization","url":"https://api.github.com/users/django"},"private":false,"pulls_url":"https://api.github.com/repos/django/djangoproject.com/pulls{/number}","pushed_at":"2019-03-16T23:41:36Z","releases_url":"https://api.github.com/repos/django/djangoproject.com/releases{/id}","size":15197,"ssh_url":"git@github.com:django/djangoproject.com.git","stargazers_count":1001,"stargazers_url":"https://api.github.com/repos/django/djangoproject.com/stargazers","statuses_url":"https://api.github.com/repos/django/djangoproject.com/statuses/{sha}","subscribers_url":"https://api.github.com/repos/django/djangoproject.com/subscribers","subscription_url":"https://api.github.com/repos/django/djangoproject.com/subscription","svn_url":"https://github.com/django/djangoproject.com","tags_url":"https://api.github.com/repos/django/djangoproject.com/tags","teams_url":"https://api.github.com/repos/django/djangoproject.com/teams","trees_url":"https://api.github.com/repos/django/djangoproject.com/git/trees{/sha}","updated_at":"2019-03-16T23:41:36Z","url":"https://api.github.com/repos/django/djangoproject.com","watchers":1001,"watchers_count":1001},"sha":"6719c7572f9e802331b16e2ff4b637d4911c2b38","user":{"avatar_url":"https://avatars2.githubusercontent.com/u/27804?v=4","events_url":"https://api.github.com/users/django/events{/privacy}","followers_url":"https://api.github.com/users/django/followers","following_url":"https://api.github.com/users/django/following{/other_user}","gists_url":"https://api.github.com/users/django/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/django","id":27804,"login":"django","node_id":"MDEyOk9yZ2FuaXphdGlvbjI3ODA0","organizations_url":"https://api.github.com/users/django/orgs","received_events_url":"https://api.github.com/users/django/received_events","repos_url":"https://api.github.com/users/django/repos","site_admin":false,"starred_url":"https://api.github.com/users/django/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/django/subscriptions","type":"Organization","url":"https://api.github.com/users/django"}},"body":"forgot to import socket before using it in the previous patch. >.<\n","changed_files":1,"closed_at":"2011-07-04T15:23:31Z","comments":1,"comments_url":"https://api.github.com/repos/django/djangoproject.com/issues/17/comments","commits":1,"commits_data":["f397b881184a14510fb5a8ef1dd72732839fd286"],"commits_url":"https://api.github.com/repos/django/djangoproject.com/pulls/17/commits","created_at":"2011-06-24T23:31:08Z","deletions":0,"diff_url":"https://github.com/django/djangoproject.com/pull/17.diff","head":{"label":"PaulMcMillan:patch-1","ref":"patch-1","repo":null,"sha":"f397b881184a14510fb5a8ef1dd72732839fd286","user":{"avatar_url":"https://avatars2.githubusercontent.com/u/494159?v=4","events_url":"https://api.github.com/users/PaulMcMillan/events{/privacy}","followers_url":"https://api.github.com/users/PaulMcMillan/followers","following_url":"https://api.github.com/users/PaulMcMillan/following{/other_user}","gists_url":"https://api.github.com/users/PaulMcMillan/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/PaulMcMillan","id":494159,"login":"PaulMcMillan","node_id":"MDQ6VXNlcjQ5NDE1OQ==","organizations_url":"https://api.github.com/users/PaulMcMillan/orgs","received_events_url":"https://api.github.com/users/PaulMcMillan/received_events","repos_url":"https://api.github.com/users/PaulMcMillan/repos","site_admin":false,"starred_url":"https://api.github.com/users/PaulMcMillan/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/PaulMcMillan/subscriptions","type":"User","url":"https://api.github.com/users/PaulMcMillan"}},"html_url":"https://github.com/django/djangoproject.com/pull/17","id":197274,"issue_url":"https://api.github.com/repos/django/djangoproject.com/issues/17","labels":[],"locked":false,"maintainer_can_modify":false,"merge_commit_sha":"","mergeable":false,"mergeable_state":"dirty","merged":false,"merged_at":null,"merged_by":null,"merged_by_data":[],"milestone":null,"node_id":"MDExOlB1bGxSZXF1ZXN0MTk3Mjc0","number":17,"patch_url":"https://github.com/django/djangoproject.com/pull/17.patch","rebaseable":false,"requested_reviewers":[],"requested_reviewers_data":[],"requested_teams":[],"review_comment_url":"https://api.github.com/repos/django/djangoproject.com/pulls/comments{/number}","review_comments":0,"review_comments_data":{},"review_comments_url":"https://api.github.com/repos/django/djangoproject.com/pulls/17/comments","state":"closed","statuses_url":"https://api.github.com/repos/django/djangoproject.com/statuses/f397b881184a14510fb5a8ef1dd72732839fd286","title":"forgot to import socket","updated_at":"2011-07-04T15:23:31Z","url":"https://api.github.com/repos/django/djangoproject.com/pulls/17","user":{"avatar_url":"https://avatars2.githubusercontent.com/u/494159?v=4","events_url":"https://api.github.com/users/PaulMcMillan/events{/privacy}","followers_url":"https://api.github.com/users/PaulMcMillan/followers","following_url":"https://api.github.com/users/PaulMcMillan/following{/other_user}","gists_url":"https://api.github.com/users/PaulMcMillan/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/PaulMcMillan","id":494159,"login":"PaulMcMillan","node_id":"MDQ6VXNlcjQ5NDE1OQ==","organizations_url":"https://api.github.com/users/PaulMcMillan/orgs","received_events_url":"https://api.github.com/users/PaulMcMillan/received_events","repos_url":"https://api.github.com/users/PaulMcMillan/repos","site_admin":false,"starred_url":"https://api.github.com/users/PaulMcMillan/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/PaulMcMillan/subscriptions","type":"User","url":"https://api.github.com/users/PaulMcMillan"},"user_data":{"avatar_url":"https://avatars2.githubusercontent.com/u/494159?v=4","bio":null,"blog":"https://twitter.com/PaulM","company":null,"created_at":"2010-11-23T23:33:46Z","email":null,"events_url":"https://api.github.com/users/PaulMcMillan/events{/privacy}","followers":119,"followers_url":"https://api.github.com/users/PaulMcMillan/followers","following":0,"following_url":"https://api.github.com/users/PaulMcMillan/following{/other_user}","gists_url":"https://api.github.com/users/PaulMcMillan/gists{/gist_id}","gravatar_id":"","hireable":null,"html_url":"https://github.com/PaulMcMillan","id":494159,"location":"SF Bay Area","login":"PaulMcMillan","name":"Paul McMillan","node_id":"MDQ6VXNlcjQ5NDE1OQ==","organizations":[{"avatar_url":"https://avatars3.githubusercontent.com/u/706131?v=4","description":null,"events_url":"https://api.github.com/orgs/zerocoordinate/events","hooks_url":"https://api.github.com/orgs/zerocoordinate/hooks","id":706131,"issues_url":"https://api.github.com/orgs/zerocoordinate/issues","login":"zerocoordinate","members_url":"https://api.github.com/orgs/zerocoordinate/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjcwNjEzMQ==","public_members_url":"https://api.github.com/orgs/zerocoordinate/public_members{/member}","repos_url":"https://api.github.com/orgs/zerocoordinate/repos","url":"https://api.github.com/orgs/zerocoordinate"}],"organizations_url":"https://api.github.com/users/PaulMcMillan/orgs","public_gists":4,"public_repos":34,"received_events_url":"https://api.github.com/users/PaulMcMillan/received_events","repos_url":"https://api.github.com/users/PaulMcMillan/repos","site_admin":false,"starred_url":"https://api.github.com/users/PaulMcMillan/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/PaulMcMillan/subscriptions","type":"User","updated_at":"2019-01-10T23:30:20Z","url":"https://api.github.com/users/PaulMcMillan"}},"origin":"https://github.com/django/djangoproject.com","perceval_version":"0.12.3","tag":"https://github.com/django/djangoproject.com","timestamp":1552904199.007064,"updated_on":1309793011.0,"uuid":"9348cb0cbe4f053e5165140c2598f4374b796276"}'

b = '{"backend_name":"GitHub","backend_version":"0.17.4","category":"pull_request","data":{"_links":{"comments":{"href":"https://api.github.com/repos/django/djangoproject.com/issues/18/comments"},"commits":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/18/commits"},"html":{"href":"https://github.com/django/djangoproject.com/pull/18"},"issue":{"href":"https://api.github.com/repos/django/djangoproject.com/issues/18"},"review_comment":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/comments{/number}"},"review_comments":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/18/comments"},"self":{"href":"https://api.github.com/repos/django/djangoproject.com/pulls/18"},"statuses":{"href":"https://api.github.com/repos/django/djangoproject.com/statuses/4e3368e1b2003f5c5bcb31d222af4b800972801f"}},"additions":1,"assignee":null,"assignees":[],"author_association":"CONTRIBUTOR","base":{"label":"django:master","ref":"master","repo":{"archive_url":"https://api.github.com/repos/django/djangoproject.com/{archive_format}{/ref}","archived":false,"assignees_url":"https://api.github.com/repos/django/djangoproject.com/assignees{/user}","blobs_url":"https://api.github.com/repos/django/djangoproject.com/git/blobs{/sha}","branches_url":"https://api.github.com/repos/django/djangoproject.com/branches{/branch}","clone_url":"https://github.com/django/djangoproject.com.git","collaborators_url":"https://api.github.com/repos/django/djangoproject.com/collaborators{/collaborator}","comments_url":"https://api.github.com/repos/django/djangoproject.com/comments{/number}","commits_url":"https://api.github.com/repos/django/djangoproject.com/commits{/sha}","compare_url":"https://api.github.com/repos/django/djangoproject.com/compare/{base}...{head}","contents_url":"https://api.github.com/repos/django/djangoproject.com/contents/{+path}","contributors_url":"https://api.github.com/repos/django/djangoproject.com/contributors","created_at":"2010-05-28T11:27:09Z","default_branch":"master","deployments_url":"https://api.github.com/repos/django/djangoproject.com/deployments","description":"Source code to djangoproject.com","downloads_url":"https://api.github.com/repos/django/djangoproject.com/downloads","events_url":"https://api.github.com/repos/django/djangoproject.com/events","fork":false,"forks":478,"forks_count":478,"forks_url":"https://api.github.com/repos/django/djangoproject.com/forks","full_name":"django/djangoproject.com","git_commits_url":"https://api.github.com/repos/django/djangoproject.com/git/commits{/sha}","git_refs_url":"https://api.github.com/repos/django/djangoproject.com/git/refs{/sha}","git_tags_url":"https://api.github.com/repos/django/djangoproject.com/git/tags{/sha}","git_url":"git://github.com/django/djangoproject.com.git","has_downloads":true,"has_issues":true,"has_pages":false,"has_projects":false,"has_wiki":false,"homepage":"https://www.djangoproject.com/","hooks_url":"https://api.github.com/repos/django/djangoproject.com/hooks","html_url":"https://github.com/django/djangoproject.com","id":690903,"issue_comment_url":"https://api.github.com/repos/django/djangoproject.com/issues/comments{/number}","issue_events_url":"https://api.github.com/repos/django/djangoproject.com/issues/events{/number}","issues_url":"https://api.github.com/repos/django/djangoproject.com/issues{/number}","keys_url":"https://api.github.com/repos/django/djangoproject.com/keys{/key_id}","labels_url":"https://api.github.com/repos/django/djangoproject.com/labels{/name}","language":"PostScript","languages_url":"https://api.github.com/repos/django/djangoproject.com/languages","license":{"key":"other","name":"Other","node_id":"MDc6TGljZW5zZTA=","spdx_id":"NOASSERTION","url":null},"merges_url":"https://api.github.com/repos/django/djangoproject.com/merges","milestones_url":"https://api.github.com/repos/django/djangoproject.com/milestones{/number}","mirror_url":null,"name":"djangoproject.com","node_id":"MDEwOlJlcG9zaXRvcnk2OTA5MDM=","notifications_url":"https://api.github.com/repos/django/djangoproject.com/notifications{?since,all,participating}","open_issues":46,"open_issues_count":46,"owner":{"avatar_url":"https://avatars2.githubusercontent.com/u/27804?v=4","events_url":"https://api.github.com/users/django/events{/privacy}","followers_url":"https://api.github.com/users/django/followers","following_url":"https://api.github.com/users/django/following{/other_user}","gists_url":"https://api.github.com/users/django/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/django","id":27804,"login":"django","node_id":"MDEyOk9yZ2FuaXphdGlvbjI3ODA0","organizations_url":"https://api.github.com/users/django/orgs","received_events_url":"https://api.github.com/users/django/received_events","repos_url":"https://api.github.com/users/django/repos","site_admin":false,"starred_url":"https://api.github.com/users/django/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/django/subscriptions","type":"Organization","url":"https://api.github.com/users/django"},"private":false,"pulls_url":"https://api.github.com/repos/django/djangoproject.com/pulls{/number}","pushed_at":"2019-03-16T23:41:36Z","releases_url":"https://api.github.com/repos/django/djangoproject.com/releases{/id}","size":15197,"ssh_url":"git@github.com:django/djangoproject.com.git","stargazers_count":1001,"stargazers_url":"https://api.github.com/repos/django/djangoproject.com/stargazers","statuses_url":"https://api.github.com/repos/django/djangoproject.com/statuses/{sha}","subscribers_url":"https://api.github.com/repos/django/djangoproject.com/subscribers","subscription_url":"https://api.github.com/repos/django/djangoproject.com/subscription","svn_url":"https://github.com/django/djangoproject.com","tags_url":"https://api.github.com/repos/django/djangoproject.com/tags","teams_url":"https://api.github.com/repos/django/djangoproject.com/teams","trees_url":"https://api.github.com/repos/django/djangoproject.com/git/trees{/sha}","updated_at":"2019-03-16T23:41:36Z","url":"https://api.github.com/repos/django/djangoproject.com","watchers":1001,"watchers_count":1001},"sha":"d8016caa490f4194ae706b78b4ada7ae20894ddf","user":{"avatar_url":"https://avatars2.githubusercontent.com/u/27804?v=4","events_url":"https://api.github.com/users/django/events{/privacy}","followers_url":"https://api.github.com/users/django/followers","following_url":"https://api.github.com/users/django/following{/other_user}","gists_url":"https://api.github.com/users/django/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/django","id":27804,"login":"django","node_id":"MDEyOk9yZ2FuaXphdGlvbjI3ODA0","organizations_url":"https://api.github.com/users/django/orgs","received_events_url":"https://api.github.com/users/django/received_events","repos_url":"https://api.github.com/users/django/repos","site_admin":false,"starred_url":"https://api.github.com/users/django/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/django/subscriptions","type":"Organization","url":"https://api.github.com/users/django"}},"body":"I just released django-push 0.4, it registers a proper admin and integrates all the fixes and improvements that were only available with the git branch. Should be cleaner this way :)\n","changed_files":2,"closed_at":"2011-07-04T15:17:54Z","comments":0,"comments_url":"https://api.github.com/repos/django/djangoproject.com/issues/18/comments","commits":1,"commits_data":["4e3368e1b2003f5c5bcb31d222af4b800972801f"],"commits_url":"https://api.github.com/repos/django/djangoproject.com/pulls/18/commits","created_at":"2011-06-30T19:55:08Z","deletions":10,"diff_url":"https://github.com/django/djangoproject.com/pull/18.diff","head":{"label":"brutasse:master","ref":"master","repo":null,"sha":"4e3368e1b2003f5c5bcb31d222af4b800972801f","user":{"avatar_url":"https://avatars0.githubusercontent.com/u/31269?v=4","events_url":"https://api.github.com/users/brutasse/events{/privacy}","followers_url":"https://api.github.com/users/brutasse/followers","following_url":"https://api.github.com/users/brutasse/following{/other_user}","gists_url":"https://api.github.com/users/brutasse/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/brutasse","id":31269,"login":"brutasse","node_id":"MDQ6VXNlcjMxMjY5","organizations_url":"https://api.github.com/users/brutasse/orgs","received_events_url":"https://api.github.com/users/brutasse/received_events","repos_url":"https://api.github.com/users/brutasse/repos","site_admin":false,"starred_url":"https://api.github.com/users/brutasse/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/brutasse/subscriptions","type":"User","url":"https://api.github.com/users/brutasse"}},"html_url":"https://github.com/django/djangoproject.com/pull/18","id":205005,"issue_url":"https://api.github.com/repos/django/djangoproject.com/issues/18","labels":[],"locked":false,"maintainer_can_modify":false,"merge_commit_sha":"a0a7564cd8aa559ba6dca30cd83f6c6e9f8cb6fc","mergeable":null,"mergeable_state":"unknown","merged":true,"merged_at":"2011-07-04T15:17:54Z","merged_by":{"avatar_url":"https://avatars1.githubusercontent.com/u/21148?v=4","events_url":"https://api.github.com/users/jacobian/events{/privacy}","followers_url":"https://api.github.com/users/jacobian/followers","following_url":"https://api.github.com/users/jacobian/following{/other_user}","gists_url":"https://api.github.com/users/jacobian/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/jacobian","id":21148,"login":"jacobian","node_id":"MDQ6VXNlcjIxMTQ4","organizations_url":"https://api.github.com/users/jacobian/orgs","received_events_url":"https://api.github.com/users/jacobian/received_events","repos_url":"https://api.github.com/users/jacobian/repos","site_admin":false,"starred_url":"https://api.github.com/users/jacobian/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/jacobian/subscriptions","type":"User","url":"https://api.github.com/users/jacobian"},"merged_by_data":{"avatar_url":"https://avatars1.githubusercontent.com/u/21148?v=4","bio":null,"blog":"http://jacobian.org/","company":null,"created_at":"2008-08-19T15:20:56Z","email":null,"events_url":"https://api.github.com/users/jacobian/events{/privacy}","followers":2343,"followers_url":"https://api.github.com/users/jacobian/followers","following":23,"following_url":"https://api.github.com/users/jacobian/following{/other_user}","gists_url":"https://api.github.com/users/jacobian/gists{/gist_id}","gravatar_id":"","hireable":null,"html_url":"https://github.com/jacobian","id":21148,"location":"Davis, CA","login":"jacobian","name":"Jacob Kaplan-Moss","node_id":"MDQ6VXNlcjIxMTQ4","organizations":[{"avatar_url":"https://avatars2.githubusercontent.com/u/27804?v=4","description":"","events_url":"https://api.github.com/orgs/django/events","hooks_url":"https://api.github.com/orgs/django/hooks","id":27804,"issues_url":"https://api.github.com/orgs/django/issues","login":"django","members_url":"https://api.github.com/orgs/django/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjI3ODA0","public_members_url":"https://api.github.com/orgs/django/public_members{/member}","repos_url":"https://api.github.com/orgs/django/repos","url":"https://api.github.com/orgs/django"},{"avatar_url":"https://avatars1.githubusercontent.com/u/308096?v=4","description":"Django, Python, PostgreSQL, Kubernetes and Ops. We offer technical support and consultation for complex open source systems.","events_url":"https://api.github.com/orgs/revsys/events","hooks_url":"https://api.github.com/orgs/revsys/hooks","id":308096,"issues_url":"https://api.github.com/orgs/revsys/issues","login":"revsys","members_url":"https://api.github.com/orgs/revsys/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjMwODA5Ng==","public_members_url":"https://api.github.com/orgs/revsys/public_members{/member}","repos_url":"https://api.github.com/orgs/revsys/repos","url":"https://api.github.com/orgs/revsys"}],"organizations_url":"https://api.github.com/users/jacobian/orgs","public_gists":133,"public_repos":39,"received_events_url":"https://api.github.com/users/jacobian/received_events","repos_url":"https://api.github.com/users/jacobian/repos","site_admin":false,"starred_url":"https://api.github.com/users/jacobian/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/jacobian/subscriptions","type":"User","updated_at":"2019-03-02T13:13:14Z","url":"https://api.github.com/users/jacobian"},"milestone":null,"node_id":"MDExOlB1bGxSZXF1ZXN0MjA1MDA1","number":18,"patch_url":"https://github.com/django/djangoproject.com/pull/18.patch","rebaseable":null,"requested_reviewers":[],"requested_reviewers_data":[],"requested_teams":[],"review_comment_url":"https://api.github.com/repos/django/djangoproject.com/pulls/comments{/number}","review_comments":0,"review_comments_data":{},"review_comments_url":"https://api.github.com/repos/django/djangoproject.com/pulls/18/comments","state":"closed","statuses_url":"https://api.github.com/repos/django/djangoproject.com/statuses/4e3368e1b2003f5c5bcb31d222af4b800972801f","title":"Update to a properly packaged django-push","updated_at":"2011-07-04T15:17:54Z","url":"https://api.github.com/repos/django/djangoproject.com/pulls/18","user":{"avatar_url":"https://avatars0.githubusercontent.com/u/31269?v=4","events_url":"https://api.github.com/users/brutasse/events{/privacy}","followers_url":"https://api.github.com/users/brutasse/followers","following_url":"https://api.github.com/users/brutasse/following{/other_user}","gists_url":"https://api.github.com/users/brutasse/gists{/gist_id}","gravatar_id":"","html_url":"https://github.com/brutasse","id":31269,"login":"brutasse","node_id":"MDQ6VXNlcjMxMjY5","organizations_url":"https://api.github.com/users/brutasse/orgs","received_events_url":"https://api.github.com/users/brutasse/received_events","repos_url":"https://api.github.com/users/brutasse/repos","site_admin":false,"starred_url":"https://api.github.com/users/brutasse/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/brutasse/subscriptions","type":"User","url":"https://api.github.com/users/brutasse"},"user_data":{"avatar_url":"https://avatars0.githubusercontent.com/u/31269?v=4","bio":"VP of engineering at @exoscale. Pythonista, Clojurian, Djangonaut, Javascripter.","blog":"https://bruno.renie.fr","company":"@exoscale ","created_at":"2008-10-27T16:10:20Z","email":null,"events_url":"https://api.github.com/users/brutasse/events{/privacy}","followers":200,"followers_url":"https://api.github.com/users/brutasse/followers","following":4,"following_url":"https://api.github.com/users/brutasse/following{/other_user}","gists_url":"https://api.github.com/users/brutasse/gists{/gist_id}","gravatar_id":"","hireable":null,"html_url":"https://github.com/brutasse","id":31269,"location":"Lausanne area, Switzerland","login":"brutasse","name":"Bruno Reni\u00e9","node_id":"MDQ6VXNlcjMxMjY5","organizations":[{"avatar_url":"https://avatars3.githubusercontent.com/u/467182?v=4","description":null,"events_url":"https://api.github.com/orgs/mobilenuggets/events","hooks_url":"https://api.github.com/orgs/mobilenuggets/hooks","id":467182,"issues_url":"https://api.github.com/orgs/mobilenuggets/issues","login":"mobilenuggets","members_url":"https://api.github.com/orgs/mobilenuggets/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjQ2NzE4Mg==","public_members_url":"https://api.github.com/orgs/mobilenuggets/public_members{/member}","repos_url":"https://api.github.com/orgs/mobilenuggets/repos","url":"https://api.github.com/orgs/mobilenuggets"},{"avatar_url":"https://avatars0.githubusercontent.com/u/1714806?v=4","description":"","events_url":"https://api.github.com/orgs/graphite-project/events","hooks_url":"https://api.github.com/orgs/graphite-project/hooks","id":1714806,"issues_url":"https://api.github.com/orgs/graphite-project/issues","login":"graphite-project","members_url":"https://api.github.com/orgs/graphite-project/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjE3MTQ4MDY=","public_members_url":"https://api.github.com/orgs/graphite-project/public_members{/member}","repos_url":"https://api.github.com/orgs/graphite-project/repos","url":"https://api.github.com/orgs/graphite-project"},{"avatar_url":"https://avatars2.githubusercontent.com/u/1738279?v=4","description":null,"events_url":"https://api.github.com/orgs/feedhq/events","hooks_url":"https://api.github.com/orgs/feedhq/hooks","id":1738279,"issues_url":"https://api.github.com/orgs/feedhq/issues","login":"feedhq","members_url":"https://api.github.com/orgs/feedhq/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjE3MzgyNzk=","public_members_url":"https://api.github.com/orgs/feedhq/public_members{/member}","repos_url":"https://api.github.com/orgs/feedhq/repos","url":"https://api.github.com/orgs/feedhq"},{"avatar_url":"https://avatars3.githubusercontent.com/u/2023286?v=4","description":"","events_url":"https://api.github.com/orgs/exoscale/events","hooks_url":"https://api.github.com/orgs/exoscale/hooks","id":2023286,"issues_url":"https://api.github.com/orgs/exoscale/issues","login":"exoscale","members_url":"https://api.github.com/orgs/exoscale/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjIwMjMyODY=","public_members_url":"https://api.github.com/orgs/exoscale/public_members{/member}","repos_url":"https://api.github.com/orgs/exoscale/repos","url":"https://api.github.com/orgs/exoscale"},{"avatar_url":"https://avatars0.githubusercontent.com/u/6770217?v=4","description":null,"events_url":"https://api.github.com/orgs/brutasse-archive/events","hooks_url":"https://api.github.com/orgs/brutasse-archive/hooks","id":6770217,"issues_url":"https://api.github.com/orgs/brutasse-archive/issues","login":"brutasse-archive","members_url":"https://api.github.com/orgs/brutasse-archive/members{/member}","node_id":"MDEyOk9yZ2FuaXphdGlvbjY3NzAyMTc=","public_members_url":"https://api.github.com/orgs/brutasse-archive/public_members{/member}","repos_url":"https://api.github.com/orgs/brutasse-archive/repos","url":"https://api.github.com/orgs/brutasse-archive"}],"organizations_url":"https://api.github.com/users/brutasse/orgs","public_gists":13,"public_repos":47,"received_events_url":"https://api.github.com/users/brutasse/received_events","repos_url":"https://api.github.com/users/brutasse/repos","site_admin":false,"starred_url":"https://api.github.com/users/brutasse/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/brutasse/subscriptions","type":"User","updated_at":"2018-11-05T08:06:43Z","url":"https://api.github.com/users/brutasse"}},"origin":"https://github.com/django/djangoproject.com","perceval_version":"0.12.3","tag":"https://github.com/django/djangoproject.com","timestamp":1552904197.859574,"updated_on":1309792674.0,"uuid":"74040fd2110fd9540d4e5dddf720d9f1cee726c5"}'

print(a == b)

False
